Tạo kết nối với drive

# Tạo kết nối drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Import các thư viện

In [2]:
import numpy  as np
import string 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# Xử lý data

In [3]:

file=open('/content/drive/MyDrive/IT_documents_University/Subjects/F3_semester_ThirdYear/DeepLearning/WordPrediction/data_vtc_giao_duc.txt',encoding="utf8")
raw_text=file.read()
raw_text = raw_text.lower()




In [5]:
chars = sorted(list(set(raw_text)))
print(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))


['\n', ' ', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '¾', 'à', 'á', 'â', 'ã', 'è', 'é', 'ê', 'ì', 'í', 'ò', 'ó', 'ô', 'õ', 'ù', 'ú', 'ý', 'ă', 'đ', 'ĩ', 'ũ', 'ơ', 'ư', 'ϲ', 'ạ', 'ả', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ẹ', 'ẻ', 'ẽ', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ỉ', 'ị', 'ọ', 'ỏ', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'ụ', 'ủ', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'ỳ', 'ỵ', 'ỷ', 'ỹ', '\u2003', '⅓']


In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)



Total Characters:  1364418
Total Vocab:  112


In [8]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

print ("Total Patterns: ", n_patterns)


Total Patterns:  1364318


In [10]:
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [11]:

print(X.shape)
print(y.shape)


(1364318, 100, 1)
(1364318, 112)


**văn bản in đậm**# Define model

In [12]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
#tạo checkpoint
filepath="/content/drive/MyDrive/IT_documents_University/Subjects/F3_semester_ThirdYear/DeepLearning/WordPrediction/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


# Train model

In [ ]:
H=model.fit(X, y, epochs=100, batch_size=1024, callbacks=callbacks_list)

Epoch 1/100
1333/1333 [==============================] - 308s 206ms/step - loss: 3.1861

Epoch 00001: loss improved from inf to 2.82090, saving model to /content/drive/MyDrive/IT_documents_University/Subjects/F3_semester_ThirdYear/DeepLearning/WordPrediction/weights-improvement-01-2.8209.hdf5
Epoch 2/100
1333/1333 [==============================] - 277s 208ms/step - loss: 2.3179

Epoch 00002: loss improved from 2.82090 to 2.25555, saving model to /content/drive/MyDrive/IT_documents_University/Subjects/F3_semester_ThirdYear/DeepLearning/WordPrediction/weights-improvement-02-2.2556.hdf5
Epoch 3/100
1333/1333 [==============================] - 278s 209ms/step - loss: 2.1097

Epoch 00003: loss improved from 2.25555 to 2.08000, saving model to /content/drive/MyDrive/IT_documents_University/Subjects/F3_semester_ThirdYear/DeepLearning/WordPrediction/weights-improvement-03-2.0800.hdf5
Epoch 4/100
1333/1333 [==============================] - 277s 208ms/step - loss: 1.9998

Epoch 00004: loss imp

# Vẽ đồ thị

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
fig = plt.figure()
numOfEpoch = 100
plt.plot(np.arange(0, numOfEpoch), H.history['loss'], label='training loss')
plt.plot(np.arange(0, numOfEpoch), H.history['val_loss'], label='validation loss')
plt.plot(np.arange(0, numOfEpoch), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, numOfEpoch), H.history['val_accuracy'], label='validation accuracy')
plt.title('Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss|Accuracy')
plt.legend()